In [1]:
#A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and 


In [2]:
# class pyspark.sql.SparkSession, The entry point to programming Spark with the Dataset and DataFrame API.
#A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and 
# read parquet files.To create a SparkSession, use the following builder pattern:

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("PySpark NYC Parking Tickets") \
    .getOrCreate()

In [3]:

df = spark.read.format("csv").load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string]

In [4]:
#DataFame will have columns, and we use a schema to define them.
df.printSchema()

# printSchema returns schema in tree format

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [5]:
df.schema
#Returns the schema of this DataFrame as a pyspark.sql.types.StructType.

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,StringType,true),StructField(_c5,StringType,true),StructField(_c6,StringType,true),StructField(_c7,StringType,true),StructField(_c8,StringType,true),StructField(_c9,StringType,true)))

In [6]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("Summons_Number", StringType(), True),
  StructField("Plate_ID", StringType(), True),
  StructField("Registration_State", StringType(), True),
  StructField("Issue_Date", StringType(), True),
  StructField("Violation_Code", StringType(), True),
  StructField("Vehicle_Body_Type", StringType(), True),
  StructField("Vehicle_Make", StringType(), True),
  StructField("Violation_Precinct", StringType(), True),
  StructField("Issuer_Precinct", StringType(), True),
  StructField("Violation_Time", StringType(), True)
])

df = spark.read.format("csv").schema(myManualSchema).load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [7]:
df.printSchema()

root
 |-- Summons_Number: string (nullable = true)
 |-- Plate_ID: string (nullable = true)
 |-- Registration_State: string (nullable = true)
 |-- Issue_Date: string (nullable = true)
 |-- Violation_Code: string (nullable = true)
 |-- Vehicle_Body_Type: string (nullable = true)
 |-- Vehicle_Make: string (nullable = true)
 |-- Violation_Precinct: string (nullable = true)
 |-- Issuer_Precinct: string (nullable = true)
 |-- Violation_Time: string (nullable = true)



In [8]:
#########################
# Sample Data
#########################
df.show(2)

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons_Number|Plate_ID|Registration_State|Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
|    5092469481| GZH7067|                NY|2016-07-10|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
only showing top 2 rows



In [9]:
##############################################################################################################################
# Assumption : Lots of Data Columns are having NULL but dont want to drop them since it is just EDA aand not Model Building
##############################################################################################################################

# Above equivalent sql syntax
# register DataFrame as temp table
df.createOrReplaceTempView("NYC_Table")

In [10]:
##########################################################################
# EDA 1 : 1.Find the total number of tickets for the year.
##########################################################################
# Summons_Number is taken as Ticket Number here
# Assumption that ALl Data is 2017 Only and no Year Condition is added.

DF_sql = spark.sql("SELECT count(distinct Summons_Number) as distinct_ticket_count FROM NYC_Table")

DF_sql.take(2)

[Row(distinct_ticket_count=10803029)]

In [11]:
#########################################################################################################
# 2.Find out the number of unique states from where the cars that got parking tickets came. 
# (Hint: Use the column 'Registration State'.)
# There is a numeric entry '99' in the column, which should be corrected.
# Replace it with the state having the maximum entries. Provide the number of unique states again.
#########################################################################################################


In [12]:
DF_sql = spark.sql("SELECT count(distinct Summons_Number) as ticket_count_state,Registration_State FROM NYC_Table Group By Registration_State Order By ticket_count_state DESC")

DF_sql.take(5)

[Row(ticket_count_state=8481061, Registration_State='NY'),
 Row(ticket_count_state=925965, Registration_State='NJ'),
 Row(ticket_count_state=285419, Registration_State='PA'),
 Row(ticket_count_state=144556, Registration_State='FL'),
 Row(ticket_count_state=141088, Registration_State='CT')]

In [13]:
################################################################################################################
# NY is the State with Maximum Parking Tickets as is the Case Study on New York Only :-)
################################################################################################################

In [14]:
#########################################################################
# Replace 99 with NY in the Column Registration_State
#########################################################################
replace = {'99':'NY'}
df = df.na.replace(replace,1,"Registration_State")

/usr/local/spark2.4/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [15]:
# Get the State with Maximum Entries of Summons.

# Above equivalent sql syntax
# register DataFrame as temp table
df.createOrReplaceTempView("NYC_Table")

DF_sql = spark.sql("SELECT count(distinct Summons_Number) as ticket_count_state,Registration_State FROM NYC_Table Group By Registration_State Order By ticket_count_state DESC")
DF_sql.take(5)

[Row(ticket_count_state=8517686, Registration_State='NY'),
 Row(ticket_count_state=925965, Registration_State='NJ'),
 Row(ticket_count_state=285419, Registration_State='PA'),
 Row(ticket_count_state=144556, Registration_State='FL'),
 Row(ticket_count_state=141088, Registration_State='CT')]

In [16]:
###############################################################################################
# Value of the NY is Increased from 8481061 To 8517686 and hence replacement is done
###############################################################################################

In [17]:
################################################################################################
# EDA 2 : NO of States with Parking Tickets
################################################################################################
DF_sql = spark.sql("SELECT count(distinct Registration_State) as unique_state_count FROM NYC_Table")
DF_sql.take(2)

[Row(unique_state_count=67)]

In [18]:
########################################################################
# EDA 2 : No of Unique Registration States with Parking Tickets - 67
########################################################################

In [19]:
######################################################################################################
# Aggregation Tasks
######################################################################################################

In [20]:
# 1. How often does each violation code occur? Display the frequency of the top five violation codes.

DF_sql = spark.sql("SELECT Count(Violation_Code) as violation_code_count,Violation_Code FROM NYC_Table Group By Violation_Code ORDER BY violation_code_count DESC")
DF_sql.take(5)

[Row(violation_code_count=1528588, Violation_Code='21'),
 Row(violation_code_count=1400614, Violation_Code='36'),
 Row(violation_code_count=1062304, Violation_Code='38'),
 Row(violation_code_count=893498, Violation_Code='14'),
 Row(violation_code_count=618593, Violation_Code='20')]

In [21]:
#############################################################################
# Top 5 Violation Code are 21,36,38,14,20
#############################################################################

In [22]:
## 2.How often does each 'vehicle body type' get a parking ticket? 
## How about the 'vehicle make'? 
## (Hint: Find the top 5 for both.)

In [23]:
##########
DF_sql = spark.sql("SELECT Count(Summons_Number) as violation_summon_count,Vehicle_Body_Type FROM NYC_Table Group By Vehicle_Body_Type ORDER BY violation_summon_count DESC")
DF_sql.take(5)

[Row(violation_summon_count=3719802, Vehicle_Body_Type='SUBN'),
 Row(violation_summon_count=3082020, Vehicle_Body_Type='4DSD'),
 Row(violation_summon_count=1411970, Vehicle_Body_Type='VAN'),
 Row(violation_summon_count=687330, Vehicle_Body_Type='DELV'),
 Row(violation_summon_count=438191, Vehicle_Body_Type='SDN')]

In [24]:
#############################################################################
# Top 5 Violation Body type are SUBN / 4DSD / VAN / DELV and SDN
#############################################################################

In [25]:
##########
DF_sql = spark.sql("SELECT Count(Summons_Number) as violation_summon_count,Vehicle_Make FROM NYC_Table Group By Vehicle_Make ORDER BY violation_summon_count DESC")
DF_sql.take(5)

[Row(violation_summon_count=1280958, Vehicle_Make='FORD'),
 Row(violation_summon_count=1211451, Vehicle_Make='TOYOT'),
 Row(violation_summon_count=1079238, Vehicle_Make='HONDA'),
 Row(violation_summon_count=918590, Vehicle_Make='NISSA'),
 Row(violation_summon_count=714655, Vehicle_Make='CHEVR')]

In [26]:
#############################################################################
# Top 5 Vehicle Body Type are FORD / TOYOTA / HONDA / NISSAN and Chevrolet
#############################################################################

In [27]:
###########################################################################################################
## 3.A precinct is a police station that has a certain zone of the city under its command. 
## Find the (5 highest) frequencies of tickets for each of the following:
## 1.'Violation Precinct' (This is the precinct of the zone where the violation occurred). 
## Using this, can you draw any insights for parking violations in any specific areas of the city?
## 2.	'Issuer Precinct' (This is the precinct that issued the ticket.)
## Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. 
## These are erroneous entries. Hence, you need to provide the records for five correct precincts. 
# (Hint: Print the top six entries after sorting.)
###########################################################################################################


In [28]:
########## Violation Precinct
DF_sql = spark.sql("SELECT Count(Summons_Number) as violation_summon_count,Violation_Precinct FROM NYC_Table Group By Violation_Precinct ORDER BY violation_summon_count DESC")
DF_sql.take(6)

[Row(violation_summon_count=2072400, Violation_Precinct='0'),
 Row(violation_summon_count=535671, Violation_Precinct='19'),
 Row(violation_summon_count=352450, Violation_Precinct='14'),
 Row(violation_summon_count=331810, Violation_Precinct='1'),
 Row(violation_summon_count=306920, Violation_Precinct='18'),
 Row(violation_summon_count=296514, Violation_Precinct='114')]

In [29]:
########## 
## the'Violating Precinct' as '0'are erroneous entries.
## hence top five 'Violating Precinct' are 19,14,1,18 and 114

In [30]:
########## Issuer Precinct
DF_sql = spark.sql("SELECT Count(Summons_Number) as violation_summon_count,Issuer_Precinct FROM NYC_Table Group By Issuer_Precinct ORDER BY violation_summon_count DESC")
DF_sql.take(6)

[Row(violation_summon_count=2388479, Issuer_Precinct='0'),
 Row(violation_summon_count=521513, Issuer_Precinct='19'),
 Row(violation_summon_count=344977, Issuer_Precinct='14'),
 Row(violation_summon_count=321170, Issuer_Precinct='1'),
 Row(violation_summon_count=296553, Issuer_Precinct='18'),
 Row(violation_summon_count=289950, Issuer_Precinct='114')]

In [31]:
########## 
## the 'Issuing Precinct' as '0'are erroneous entries.
## hence top five Issuing Precinct' are 19,14,1,18 and 114

In [32]:
###### 4.Find the violation code frequencies for three precincts that have issued the most number of tickets. 
## Do these precinct zones have an exceptionally high frequency of certain violation codes? 
## Are these codes common across precincts? 
## (Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)

In [33]:
########## violation code frequencies for three precincts that have issued the most number of tickets i.e. for 19,14 and 1
DF_sql = spark.sql("SELECT Count(Violation_Code) as Violation_Code_count,Violation_Precinct FROM NYC_Table Group By Violation_Precinct ORDER BY Violation_Code_count DESC")
DF_sql.take(4)

[Row(Violation_Code_count=2072400, Violation_Precinct='0'),
 Row(Violation_Code_count=535671, Violation_Precinct='19'),
 Row(Violation_Code_count=352450, Violation_Precinct='14'),
 Row(Violation_Code_count=331810, Violation_Precinct='1')]

In [34]:
########## top 3 violation codes frequency
DF_sql = spark.sql("SELECT Count(Violation_Code) as Violation_Code_count,Violation_Code FROM NYC_Table Group By Violation_Code ORDER BY Violation_Code_count DESC")
DF_sql.take(3)

[Row(Violation_Code_count=1528588, Violation_Code='21'),
 Row(Violation_Code_count=1400614, Violation_Code='36'),
 Row(Violation_Code_count=1062304, Violation_Code='38')]

In [35]:
##### Yes these precinct zones have an exceptionally high frequency of certain violation codes

In [36]:
########## top 3 violation codes for Violation_Precinct='19'
DF_sql = spark.sql("SELECT Count(Violation_Code) as Violation_Code_count,Violation_Code FROM NYC_Table where Violation_Precinct = '19' Group By Violation_Code ORDER BY Violation_Code_count DESC ")
DF_sql.take(3)

[Row(Violation_Code_count=90530, Violation_Code='46'),
 Row(Violation_Code_count=74926, Violation_Code='38'),
 Row(Violation_Code_count=73359, Violation_Code='37')]

In [37]:
########## top 3 violation codes for Violation_Precinct='14'
DF_sql = spark.sql("SELECT Count(Violation_Code) as Violation_Code_count,Violation_Code FROM NYC_Table where Violation_Precinct = '14' Group By Violation_Code ORDER BY Violation_Code_count DESC ")
DF_sql.take(3)

[Row(Violation_Code_count=75850, Violation_Code='14'),
 Row(Violation_Code_count=58032, Violation_Code='69'),
 Row(Violation_Code_count=40150, Violation_Code='31')]

In [38]:
########## top 3 violation codes for Violation_Precinct='1'
DF_sql = spark.sql("SELECT Count(Violation_Code) as Violation_Code_count,Violation_Code FROM NYC_Table where Violation_Precinct = '1' Group By Violation_Code ORDER BY Violation_Code_count DESC ")
DF_sql.take(3)

[Row(Violation_Code_count=76375, Violation_Code='14'),
 Row(Violation_Code_count=39197, Violation_Code='16'),
 Row(Violation_Code_count=28768, Violation_Code='20')]

In [39]:
###### top three codes are not common across precincts 19,14 and 1

In [40]:
###  5. Find out the properties of parking violations across different times of the day:
##Find a way to deal with missing values, if any.
#(Hint: Check for the null values using 'isNull' under the SQL. 
 #Also, to remove the null values, check the 'dropna' command in the API documentation.)

    
##The Violation Time field is specified in a strange format. 
#Find a way to make this a time attribute that you can use to divide into groups.


##Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. 
#For each of these groups, find the three most commonly occurring violations.
#(Hint: Use the CASE-WHEN in SQL view to segregate into bins. 
#To find the most commonly occurring violations, you can use an approach similar to the one mentioned in hint for question 4.)


##Now, try another direction. For the three most commonly occurring violation codes, 
#find the most common time of the day (in terms of the bins from the previous part).

In [41]:
DF_sql = spark.sql("SELECT * FROM NYC_Table WHERE ISNULL(Violation_Time)")

In [42]:
DF_sql.take(5)

[]

In [43]:
## as we getting 0 results we are go to go  

In [44]:
#### 7.The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department.
# Let’s take an example of estimating this for the three most commonly occurring codes:

## Find the total occurrences of the three most common violation codes.

## Then, visit the website:
# http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page

    ##It lists the fines associated with different violation codes. They’re divided into two categories: 
# one for the highest-density locations in the city and the other for the rest of the city. 
# For the sake of simplicity, take the average of the two.
# Using this information, find the total amount collected for the three violation codes with the maximum tickets.
# State the code that has the highest total collection.
# What can you intuitively infer from these findings?

In [45]:
## the total occurrences of the three most common violation codes.
# as we alredy discovered that 21,36 and 38 are three most common violation codes

DF_sql = spark.sql("SELECT count(Violation_Code) as Violation_Code_Sum FROM NYC_Table where Violation_Code='21' OR Violation_Code='36'OR Violation_Code='38'")
DF_sql.take(1)

[Row(Violation_Code_Sum=3991506)]

In [46]:
## amount collected for the violation code 21

DF_sql1 = spark.sql("SELECT (Count(Violation_Code)*(45+65)/2) as Violation_Code_amount FROM NYC_Table where Violation_Code='21' group by Violation_Code ")
DF_sql1.take(1)

[Row(Violation_Code_amount=84072340.0)]

In [47]:
## amount collected for the violation code 36

DF_sql2 = spark.sql("SELECT (Count(Violation_Code)*(50+50)/2) as Violation_Code_amount FROM NYC_Table where Violation_Code='36' group by Violation_Code ")
DF_sql2.take(1)

[Row(Violation_Code_amount=70030700.0)]

In [48]:
## amount collected for the violation code 38

DF_sql3 = spark.sql("SELECT (Count(Violation_Code)*(65+35)/2) as Violation_Code_amount FROM NYC_Table where Violation_Code='38' group by Violation_Code ")
DF_sql3.take(1)


[Row(Violation_Code_amount=53115200.0)]

In [49]:
## Violation_Code 21 has highest collection 
## No parking (where parking is not allowed by sign, street marking or traffic control device) is most ignored violation 

In [50]:
spark.stop()